In [5]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("update_balances.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def calculate_and_update_balances():
    """
    Рассчитывает и обновляет balance_mi8t, balance_mi17, balance_total, stock_mi8t, stock_mi17, stock_total
    для первой даты в таблице OlapCube_VNV.
    """
    try:
        # Извлечь первую дату
        query_first_date = f"""
        SELECT MIN(Dates) AS first_date
        FROM {database_name}.OlapCube_VNV
        """
        first_date_result = client.execute(query_first_date)
        first_date = first_date_result[0][0]
        logger.info(f"Первая дата в OlapCube_VNV: {first_date}")

        # Рассчитать balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Эксплуатация' AND Dates = '{first_date}'
        """
        balance_mi8t_count = client.execute(query_balance_mi8t)[0][0]

        # Получить mi8t_count
        query_mi8t_count = f"""
        SELECT mi8t_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{first_date}'
        LIMIT 1
        """
        mi8t_count = client.execute(query_mi8t_count)[0][0]
        balance_mi8t = balance_mi8t_count - mi8t_count
        logger.info(f"balance_mi8t: {balance_mi8t}")

        # Рассчитать balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Эксплуатация' AND Dates = '{first_date}'
        """
        balance_mi17_count = client.execute(query_balance_mi17)[0][0]

        # Получить mi17_count
        query_mi17_count = f"""
        SELECT mi17_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{first_date}'
        LIMIT 1
        """
        mi17_count = client.execute(query_mi17_count)[0][0]
        balance_mi17 = balance_mi17_count - mi17_count
        logger.info(f"balance_mi17: {balance_mi17}")

        # Рассчитать balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total}")

        # Рассчитать stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Исправен' AND Dates = '{first_date}'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")

        # Рассчитать stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Исправен' AND Dates = '{first_date}'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")

        # Рассчитать stock_total
        stock_total = stock_mi8t + stock_mi17
        logger.info(f"stock_total: {stock_total}")

        # Обновить таблицу OlapCube_VNV
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = {balance_mi8t},
            balance_mi17 = {balance_mi17},
            balance_total = {balance_total},
            stock_mi8t = {stock_mi8t},
            stock_mi17 = {stock_mi17},
            stock_total = {stock_total}
        WHERE Dates = '{first_date}'
        """
        client.execute(update_query)
        logger.info("Баланс и запасы успешно обновлены для первой даты.")

    except Exception as e:
        logger.error(f"Ошибка при расчете или обновлении балансов: {e}", exc_info=True)


def main():
    calculate_and_update_balances()


if __name__ == "__main__":
    main()


2024-12-02 04:35:15,962 - INFO - Первая дата в OlapCube_VNV: 2024-11-25
2024-12-02 04:35:16,001 - INFO - balance_mi8t: -11.0
2024-12-02 04:35:16,037 - INFO - balance_mi17: 17.0
2024-12-02 04:35:16,038 - INFO - balance_total: 6.0
2024-12-02 04:35:16,055 - INFO - stock_mi8t: 3
2024-12-02 04:35:16,072 - INFO - stock_mi17: 5
2024-12-02 04:35:16,073 - INFO - stock_total: 8
2024-12-02 04:35:16,080 - INFO - Баланс и запасы успешно обновлены для первой даты.


In [2]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Уровень установлен на INFO для необходимого уровня детализации
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("algorithm_two.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def get_initial_date():
    """
    Получает самую раннюю дату из таблицы OlapCube_VNV.
    """
    try:
        query = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        result = client.execute(query)
        initial_date = result[0][0]
        logger.info(f"Начальная дата в OlapCube_VNV: {initial_date}")
        return initial_date
    except Exception as e:
        logger.error(f"Ошибка при получении начальной даты: {e}", exc_info=True)
        return None

def get_target_date(initial_date):
    """
    Вычисляет target_date = initial_date + 1 день.
    """
    try:
        if isinstance(initial_date, datetime):
            initial_date_date = initial_date.date()
        elif isinstance(initial_date, date):
            initial_date_date = initial_date
        elif isinstance(initial_date, str):
            initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
        else:
            logger.error(f"Неизвестный формат initial_date: {initial_date}")
            return None
        target_date = initial_date_date + timedelta(days=1)
        logger.info(f"target_date: {target_date}")
        return target_date
    except Exception as e:
        logger.error(f"Ошибка при вычислении target_date: {e}", exc_info=True)
        return None

def calculate_and_update_balances(target_date, mi8t_count, mi17_count):
    """
    Рассчитывает балансы и запасы для target_date и обновляет их в таблице OlapCube_VNV.
    
    :param target_date: Целевая дата для расчетов (datetime.date)
    :param mi8t_count: Значение mi8t_count для вычитания из баланса
    :param mi17_count: Значение mi17_count для вычитания из баланса
    """
    try:
        # Расчет balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV 
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-8Т' 
          AND Status_P = 'Эксплуатация'
        """
        mi8t_exploitation_count = client.execute(query_balance_mi8t)[0][0]
        balance_mi8t = mi8t_exploitation_count - mi8t_count
        logger.info(f"balance_mi8t: {balance_mi8t} (exploitation_count: {mi8t_exploitation_count} - mi8t_count: {mi8t_count})")
        
        # Расчет balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV 
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-17' 
          AND Status_P = 'Эксплуатация'
        """
        mi17_exploitation_count = client.execute(query_balance_mi17)[0][0]
        balance_mi17 = mi17_exploitation_count - mi17_count
        logger.info(f"balance_mi17: {balance_mi17} (exploitation_count: {mi17_exploitation_count} - mi17_count: {mi17_count})")
        
        # Расчет balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total} (balance_mi8t: {balance_mi8t} + balance_mi17: {balance_mi17})")
        
        # Расчет stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV 
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-8Т' 
          AND Status_P = 'Исправен'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")
        
        # Расчет stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV 
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-17' 
          AND Status_P = 'Исправен'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")
        
        # Расчет stock_total
        stock_total = stock_mi8t + stock_mi17
        logger.info(f"stock_total: {stock_total} (stock_mi8t: {stock_mi8t} + stock_mi17: {stock_mi17})")
        
        # Обновление расчетных полей в базе данных
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = %(balance_mi8t)s,
            balance_mi17 = %(balance_mi17)s,
            balance_total = %(balance_total)s,
            stock_mi8t = %(stock_mi8t)s,
            stock_mi17 = %(stock_mi17)s,
            stock_total = %(stock_total)s
        WHERE Dates = %(target_date)s
        """
        params = {
            'balance_mi8t': balance_mi8t,
            'balance_mi17': balance_mi17,
            'balance_total': balance_total,
            'stock_mi8t': stock_mi8t,
            'stock_mi17': stock_mi17,
            'stock_total': stock_total,
            'target_date': target_date
        }
        client.execute(update_query, params)
        logger.info(f"Баланс и запасы обновлены для даты {target_date}.")
    
    except Exception as e:
        logger.error(f"Ошибка при расчете и обновлении балансов и запасов: {e}", exc_info=True)

def main():
    # Шаг 1: Получение начальной даты
    initial_date = get_initial_date()
    if not initial_date:
        logger.error("Не удалось определить начальную дату. Завершаем выполнение.")
        return

    # Шаг 2: Вычисление target_date
    target_date = get_target_date(initial_date)
    if not target_date:
        logger.error("Не удалось определить target_date. Завершаем выполнение.")
        return

    # Шаг 3: Определение значений mi8t_count и mi17_count
    # Замените эти значения на актуальные или реализуйте их получение из других источников
    mi8t_count = 10  # Примерное значение, замените на актуальное
    mi17_count = 5   # Примерное значение, замените на актуальное
    logger.info(f"Используемые значения: mi8t_count = {mi8t_count}, mi17_count = {mi17_count}")

    # Шаг 4: Расчет и обновление балансов и запасов
    calculate_and_update_balances(target_date, mi8t_count, mi17_count)

if __name__ == "__main__":
    main()


2024-12-02 04:34:33,289 - INFO - Начальная дата в OlapCube_VNV: 2024-11-25
2024-12-02 04:34:33,290 - INFO - target_date: 2024-11-26
2024-12-02 04:34:33,291 - INFO - Используемые значения: mi8t_count = 10, mi17_count = 5
2024-12-02 04:34:33,326 - INFO - balance_mi8t: -10 (exploitation_count: 0 - mi8t_count: 10)
2024-12-02 04:34:33,353 - INFO - balance_mi17: -5 (exploitation_count: 0 - mi17_count: 5)
2024-12-02 04:34:33,354 - INFO - balance_total: -15 (balance_mi8t: -10 + balance_mi17: -5)
2024-12-02 04:34:33,373 - INFO - stock_mi8t: 3
2024-12-02 04:34:33,390 - INFO - stock_mi17: 2
2024-12-02 04:34:33,391 - INFO - stock_total: 5 (stock_mi8t: 3 + stock_mi17: 2)
2024-12-02 04:34:33,399 - INFO - Баланс и запасы обновлены для даты 2024-11-26.


In [3]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("update_balances.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def get_initial_date():
    """
    Получает самую раннюю дату из таблицы OlapCube_VNV.
    """
    try:
        query = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        result = client.execute(query)
        initial_date = result[0][0]
        logger.info(f"Начальная дата в OlapCube_VNV: {initial_date}")
        return initial_date
    except Exception as e:
        logger.error(f"Ошибка при получении начальной даты: {e}", exc_info=True)
        return None

def get_target_date(initial_date):
    """
    Вычисляет target_date = initial_date + 1 день.
    """
    try:
        if isinstance(initial_date, datetime):
            initial_date_date = initial_date.date()
        elif isinstance(initial_date, date):
            initial_date_date = initial_date
        elif isinstance(initial_date, str):
            initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
        else:
            logger.error(f"Неизвестный формат initial_date: {initial_date}")
            return None
        target_date = initial_date_date + timedelta(days=1)
        logger.info(f"target_date: {target_date}")
        return target_date
    except Exception as e:
        logger.error(f"Ошибка при вычислении target_date: {e}", exc_info=True)
        return None

def calculate_and_update_balances(target_date):
    """
    Рассчитывает и обновляет balance_mi8t, balance_mi17, balance_total,
    stock_mi8t, stock_mi17, stock_total для target_date в таблице OlapCube_VNV.
    """
    try:
        # 1. Расчет balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-8Т' 
          AND Status_P = 'Эксплуатация'
        """
        balance_mi8t_count = client.execute(query_balance_mi8t)[0][0]
        logger.info(f"balance_mi8t_count: {balance_mi8t_count}")

        # Получение mi8t_count
        query_mi8t_count = f"""
        SELECT mi8t_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi8t_count_result = client.execute(query_mi8t_count)
        if mi8t_count_result:
            mi8t_count = mi8t_count_result[0][0]
            balance_mi8t = balance_mi8t_count - mi8t_count
            logger.info(f"balance_mi8t: {balance_mi8t} (balance_mi8t_count: {balance_mi8t_count} - mi8t_count: {mi8t_count})")
        else:
            logger.warning(f"mi8t_count не найден для даты {target_date}. Устанавливаем balance_mi8t = balance_mi8t_count.")
            balance_mi8t = balance_mi8t_count

        # 2. Расчет balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-17' 
          AND Status_P = 'Эксплуатация'
        """
        balance_mi17_count = client.execute(query_balance_mi17)[0][0]
        logger.info(f"balance_mi17_count: {balance_mi17_count}")

        # Получение mi17_count
        query_mi17_count = f"""
        SELECT mi17_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi17_count_result = client.execute(query_mi17_count)
        if mi17_count_result:
            mi17_count = mi17_count_result[0][0]
            balance_mi17 = balance_mi17_count - mi17_count
            logger.info(f"balance_mi17: {balance_mi17} (balance_mi17_count: {balance_mi17_count} - mi17_count: {mi17_count})")
        else:
            logger.warning(f"mi17_count не найден для даты {target_date}. Устанавливаем balance_mi17 = balance_mi17_count.")
            balance_mi17 = balance_mi17_count

        # 3. Расчет balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total} (balance_mi8t: {balance_mi8t} + balance_mi17: {balance_mi17})")

        # 4. Расчет stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-8Т' 
          AND Status_P = 'Исправен'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")

        # 5. Расчет stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}' 
          AND ac_typ = 'Ми-17' 
          AND Status_P = 'Исправен'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")

        # 6. Расчет stock_total
        stock_total = stock_mi8t + stock_mi17
        logger.info(f"stock_total: {stock_total} (stock_mi8t: {stock_mi8t} + stock_mi17: {stock_mi17})")

        # Обновление расчетных полей в базе данных
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = {balance_mi8t},
            balance_mi17 = {balance_mi17},
            balance_total = {balance_total},
            stock_mi8t = {stock_mi8t},
            stock_mi17 = {stock_mi17},
            stock_total = {stock_total}
        WHERE Dates = '{target_date}'
        """
        client.execute(update_query)
        logger.info(f"Баланс и запасы успешно обновлены для даты {target_date}.")

    def main():
        # Шаг 1: Получение начальной даты
        initial_date = get_initial_date()
        if not initial_date:
            logger.error("Не удалось определить начальную дату. Завершаем выполнение.")
            return

        # Шаг 2: Вычисление target_date
        target_date = get_target_date(initial_date)
        if not target_date:
            logger.error("Не удалось определить target_date. Завершаем выполнение.")
            return

        # Шаг 3: Расчет и обновление балансов и запасов
        calculate_and_update_balances(target_date)

    if __name__ == "__main__":
        main()


SyntaxError: expected 'except' or 'finally' block (1973738597.py, line 180)

In [4]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("update_balances.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def calculate_and_update_balances():
    """
    Рассчитывает и обновляет balance_mi8t, balance_mi17, balance_total,
    stock_mi8t, stock_mi17, stock_total для target_date = initial_date + 1 день
    в таблице OlapCube_VNV.
    """
    try:
        # Извлечь initial_date (самая ранняя дата)
        query_initial_date = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        initial_date_result = client.execute(query_initial_date)
        initial_date = initial_date_result[0][0]
        logger.info(f"Initial date in OlapCube_VNV: {initial_date}")

        # Проверка наличия initial_date
        if not initial_date:
            logger.error("Не удалось получить initial_date. Завершаем выполнение.")
            return

        # Вычислить target_date = initial_date + 1 день
        if isinstance(initial_date, datetime):
            initial_date_date = initial_date.date()
        elif isinstance(initial_date, date):
            initial_date_date = initial_date
        elif isinstance(initial_date, str):
            try:
                initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
            except ValueError:
                logger.error(f"Неправильный формат initial_date: {initial_date}")
                return
        else:
            logger.error(f"Неизвестный формат initial_date: {initial_date}")
            return

        target_date = initial_date_date + timedelta(days=1)
        logger.info(f"Target date: {target_date}")

        # Рассчитать balance_mi8t
        query_balance_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi8t_count = client.execute(query_balance_mi8t)[0][0]

        # Получить mi8t_count
        query_mi8t_count = f"""
        SELECT mi8t_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi8t_count_result = client.execute(query_mi8t_count)
        if mi8t_count_result:
            mi8t_count = mi8t_count_result[0][0]
            balance_mi8t = balance_mi8t_count - mi8t_count
            logger.info(f"balance_mi8t: {balance_mi8t} (balance_mi8t_count: {balance_mi8t_count} - mi8t_count: {mi8t_count})")
        else:
            logger.warning(f"mi8t_count не найден для даты {target_date}. Устанавливаем balance_mi8t = balance_mi8t_count.")
            balance_mi8t = balance_mi8t_count

        # Рассчитать balance_mi17
        query_balance_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Эксплуатация' AND Dates = '{target_date}'
        """
        balance_mi17_count = client.execute(query_balance_mi17)[0][0]

        # Получить mi17_count
        query_mi17_count = f"""
        SELECT mi17_count 
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        mi17_count_result = client.execute(query_mi17_count)
        if mi17_count_result:
            mi17_count = mi17_count_result[0][0]
            balance_mi17 = balance_mi17_count - mi17_count
            logger.info(f"balance_mi17: {balance_mi17} (balance_mi17_count: {balance_mi17_count} - mi17_count: {mi17_count})")
        else:
            logger.warning(f"mi17_count не найден для даты {target_date}. Устанавливаем balance_mi17 = balance_mi17_count.")
            balance_mi17 = balance_mi17_count

        # Рассчитать balance_total
        balance_total = balance_mi8t + balance_mi17
        logger.info(f"balance_total: {balance_total} (balance_mi8t: {balance_mi8t} + balance_mi17: {balance_mi17})")

        # Рассчитать stock_mi8t
        query_stock_mi8t = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-8Т' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi8t = client.execute(query_stock_mi8t)[0][0]
        logger.info(f"stock_mi8t: {stock_mi8t}")

        # Рассчитать stock_mi17
        query_stock_mi17 = f"""
        SELECT COUNT(*) 
        FROM {database_name}.OlapCube_VNV
        WHERE ac_typ = 'Ми-17' AND Status_P = 'Исправен' AND Dates = '{target_date}'
        """
        stock_mi17 = client.execute(query_stock_mi17)[0][0]
        logger.info(f"stock_mi17: {stock_mi17}")

        # Рассчитать stock_total
        stock_total = stock_mi8t + stock_mi17
        logger.info(f"stock_total: {stock_total} (stock_mi8t: {stock_mi8t} + stock_mi17: {stock_mi17})")

        # Обновить таблицу OlapCube_VNV
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            balance_mi8t = {balance_mi8t},
            balance_mi17 = {balance_mi17},
            balance_total = {balance_total},
            stock_mi8t = {stock_mi8t},
            stock_mi17 = {stock_mi17},
            stock_total = {stock_total}
        WHERE Dates = '{target_date}'
        """
        client.execute(update_query)
        logger.info(f"Баланс и запасы успешно обновлены для даты {target_date}.")

    except Exception as e:
        logger.error(f"Ошибка при расчете или обновлении балансов: {e}", exc_info=True)


def main():
    calculate_and_update_balances()


if __name__ == "__main__":
    main()


2024-12-02 04:34:47,583 - INFO - Initial date in OlapCube_VNV: 2024-11-25
2024-12-02 04:34:47,584 - INFO - Target date: 2024-11-26
2024-12-02 04:34:47,622 - INFO - balance_mi8t: -86.0 (balance_mi8t_count: 0 - mi8t_count: 86.0)
2024-12-02 04:34:47,654 - INFO - balance_mi17: -70.0 (balance_mi17_count: 0 - mi17_count: 70.0)
2024-12-02 04:34:47,655 - INFO - balance_total: -156.0 (balance_mi8t: -86.0 + balance_mi17: -70.0)
2024-12-02 04:34:47,672 - INFO - stock_mi8t: 3
2024-12-02 04:34:47,690 - INFO - stock_mi17: 2
2024-12-02 04:34:47,691 - INFO - stock_total: 5 (stock_mi8t: 3 + stock_mi17: 2)
2024-12-02 04:34:47,699 - INFO - Баланс и запасы успешно обновлены для даты 2024-11-26.
